<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/alice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Colab-specific setup (use a CodeSpace to avoid the need for this).
try:
  %env COLAB_RELEASE_TAG
except:
  pass  # Not running in colab.
else:
  %pip install --ignore-requires-python --requirement 'https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.in' 'git+https://github.com/google-deepmind/concordia.git#egg=gdm-concordia'
  %pip list

In [ ]:
# @title Imports

from concordia.contrib import language_models as language_model_utils
import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs
from concordia.prefabs.simulation import generic as simulation
from concordia.typing import prefab as prefab_lib
from concordia.typing import entity as entity_lib
from concordia.typing import scene as scene_lib
from concordia.utils import helper_functions
from IPython import display
import numpy as np
import sentence_transformers
from concordia.environment.engines import simultaneous

In [ ]:
# @title Language Model Selection: provide key or select DISABLE_LANGUAGE_MODEL

# By default this colab uses models via an external API so you must provide an
# API key. TogetherAI offers open weights models from all sources.

API_KEY = ''  # @param {type: 'string'}
# See concordia/language_model/utils.py
API_TYPE = 'openai'  # e.g. 'together_ai' or 'openai'.
MODEL_NAME = (  # for API_TYPE = 'together_ai', we recommend MODEL_NAME = 'google/gemma-3-27b-it'
    'gpt-5'
)
# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
DISABLE_LANGUAGE_MODEL = False

In [ ]:
# @title Use the selected language model

# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.

if not DISABLE_LANGUAGE_MODEL and not API_KEY:
  raise ValueError('API_KEY is required.')

model = language_model_utils.language_model_setup(
    api_type=API_TYPE,
    model_name=MODEL_NAME,
    api_key=API_KEY,
    disable_language_model=DISABLE_LANGUAGE_MODEL,
)

In [ ]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = lambda _: np.ones(3)
else:
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2')
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [ ]:
test = model.sample_text(
    'Is societal and technological progress like getting a clearer picture of '
    'something true and deep?')
print(test)

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
#@title Print menu of prefabs

display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

In [ ]:
# @title Define payoff functions for prisoner's dilemma

def ipd_action_to_scores(joint_action):
    """Map joint action to scores using standard PD payoffs: T=5, R=3, P=1, S=0."""
    alice_action = joint_action.get('Alice', 'defect')
    bob_action = joint_action.get('Bob', 'defect')

    if alice_action == 'cooperate' and bob_action == 'cooperate':
        return {'Alice': 3.0, 'Bob': 3.0}  # Reward (R)
    elif alice_action == 'defect' and bob_action == 'defect':
        return {'Alice': 1.0, 'Bob': 1.0}  # Punishment (P)
    elif alice_action == 'cooperate' and bob_action == 'defect':
        return {'Alice': 0.0, 'Bob': 5.0}  # Sucker (S) / Temptation (T)
    else:  # alice defects, bob cooperates
        return {'Alice': 5.0, 'Bob': 0.0}  # Temptation (T) / Sucker (S)

def ipd_scores_to_observation(scores):
    """Convert cumulative scores to observations for each player."""
    return {
        name: f"{name}'s cumulative score is now {score:.1f} points."
        for name, score in scores.items()
    }

In [ ]:
# @title Define the scene for prisoner's dilemma decisions

player_names = ['Alice', 'Bob']

# Define the decision scene with binary choice
decision_scene = scene_lib.SceneTypeSpec(
    name='decision',
    game_master_name='default rules',
    action_spec=entity_lib.choice_action_spec(
        call_to_action='Does {name} cooperate or defect?',
        options=['cooperate', 'defect'],
    ),
)

# Create the scene specification for 4 rounds of iterated prisoner's dilemma
scenes = [
    scene_lib.SceneSpec(
        scene_type=decision_scene,
        participants=player_names,
        num_rounds=4,
        premise={
            name: [
                (
                    f"{name} is playing an iterated prisoner's dilemma game. "
                    "In each round, both players simultaneously choose to either cooperate or defect. "
                    "The payoffs are: both cooperate = 3 points each, "
                    "both defect = 1 point each, "
                    "one cooperates while other defects = 0 points for cooperator, 5 points for defector. "
                    "The goal is to maximize cumulative points over all rounds."
                ),
            ]
            for name in player_names
        },
    ),
]

In [ ]:
# @title Configure instances using library components

instances = [
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'Alice',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'Bob',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='game_theoretic_and_dramaturgic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'default rules',
            'scenes': scenes,
            'action_to_scores': ipd_action_to_scores,
            'scores_to_observation': ipd_scores_to_observation,
            'acting_order': 'simultaneous',
        },
    ),
]

In [ ]:
config = prefab_lib.Config(
    default_premise=(
        "Two agents, Alice and Bob, are playing an iterated prisoner's dilemma game. "
        "In each round, both players simultaneously choose to either cooperate or defect, "
        "without knowing what the other player will do. The payoffs are as follows: "
        "If both cooperate, each gets 3 points (mutual cooperation reward). "
        "If both defect, each gets 1 point (mutual defection punishment). "
        "If one cooperates while the other defects, the cooperator gets 0 points (sucker's payoff) "
        "and the defector gets 5 points (temptation payoff). "
        "The game will be played for a few rounds, and cumulative payoffs will be tracked."
    ),
    default_max_steps=4,  # Match number of steps needed for scene to complete
    prefabs=prefabs,
    instances=instances,
)

# The simulation

In [ ]:
# @title Initialize the simulation
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
    engine=simultaneous.Simultaneous()
)

In [ ]:
# @title Run the simulation
raw_log = []
results_log = runnable_simulation.play(
    max_steps=4,  # Match number of steps needed for scene to complete
    raw_log=raw_log
)

In [ ]:
# @title Display the log
display.HTML(results_log)

```

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```